<a href="https://colab.research.google.com/github/BrahianCarrera/KaggleCompetition/blob/main/99%20-%20modelo%20soluci%C3%B3n.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
# Carga el archivo kaggle.json
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
! python -m pip install -qq kaggle
!kaggle competitions download -c udea-ai4eng-20241
!unzip udea-ai4eng-20241.zip

Saving kaggle.json to kaggle.json
 24% 5.00M/21.2M [00:00<00:00, 43.4MB/s]
100% 21.2M/21.2M [00:00<00:00, 120MB/s] 
Archive:  udea-ai4eng-20241.zip
  inflating: submission_example.csv  
  inflating: test.csv                
  inflating: train.csv               


In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import re
%matplotlib inline
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [ ]:
test_df = pd.read_csv('/content/test.csv', encoding= 'utf-8')
df = pd.read_csv('/content/train.csv', encoding= 'utf-8')

In [ ]:
category_cols = [i for i in test_df.columns if not i in test_df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))

ESTU_PRGM_ACADEMICO ['3°  CICLO PROFESIONAL NEGOCIOS INTERNACIONALES'
 'ACTIVIDAD FISICA Y DEPORTE' 'ACUICULTURA' 'ADMINISTRACION'
 'ADMINISTRACION  FINANCIERA' 'ADMINISTRACION & SERVICIO'
 'ADMINISTRACION AERONAUTICA' 'ADMINISTRACION AGROPECUARIA'
 'ADMINISTRACION AMBIENTAL'
 'ADMINISTRACION AMBIENTAL Y DE LOS RECURSOS NATURALES'
 'ADMINISTRACION BANCARIA Y FINANCIERA' 'ADMINISTRACION COMERCIAL'
 'ADMINISTRACION COMERCIAL Y DE MERCADEO'
 'ADMINISTRACION DE COMERCIO EXTERIOR' 'ADMINISTRACION DE EMPRESAS'
 'ADMINISTRACION DE EMPRESAS  Y  GESTION AMBIENTAL'
 'ADMINISTRACION DE EMPRESAS AGROINDUSTRIALES'
 'ADMINISTRACION DE EMPRESAS AGROPECUARIAS'
 'ADMINISTRACION DE EMPRESAS COMERCIALES'
 'ADMINISTRACION DE EMPRESAS EN TELECOMUNICACIONES'
 'ADMINISTRACION DE EMPRESAS TURISTICA'
 'ADMINISTRACION DE EMPRESAS TURISTICAS'
 'ADMINISTRACION DE EMPRESAS TURISTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS TURÍSTICAS Y HOTELERAS'
 'ADMINISTRACION DE EMPRESAS Y FINANZAS'
 'ADMINISTRACION DE GESTI

In [ ]:
import pandas as pd
import numpy as np
import re
from sklearn.preprocessing import OneHotEncoder

def Preprocesar(df):
    def obtener_primera_frase(cadena):
        return cadena.split(' ', 1)[0]

    def quitar_tildes(palabra):
        return re.sub(r'[ÁÉÍÓÚ]', lambda x: {'Á': 'A', 'É': 'E', 'Í': 'I', 'Ó': 'O', 'Ú': 'U'}[x.group()], palabra)

    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(obtener_primera_frase)
    df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].apply(quitar_tildes)

    reemplazos = {
        'ADMINISTRACION': ['ADMINISTRACION', 'ADMINISTRACI¿N', 'ADMINISTRACIÒN', 'ADMINISTRACIÓN', 'ADMINSITRACION', 'ADMINSITRACIÓN'],
        'ANTROPOLOGIA': ['ANTROPOLOGIA', 'ANTROPOLOGÍA'],
        'ARTES': ['ARTE', 'ARTES', 'BELLAS', 'ANIMACION'],
        'BIOINGENIERIA': ['BIOINGENIERIA', 'BIOINGENIERÍA'],
        'BIOLOGIA': ['BIOLOGIA', 'BIOLOGÍA'],
        'COMUNICACION': ['COMUNICACION', 'COMUNICACIONES', 'COMUNICACI¿N', 'COMUNICACIÓN', 'COMUNICACIÖN'],
        'CONSTRUCCION': ['CONSTRUCCION', 'CONSTRUCCIONES'],
        'CONTADURIA': ['CONTADURIA', 'CONTADURÍA'],
        'DISEÑO': ['DISENO', 'DISE¿O', 'DISE¿¿O', 'DISEÑO'],
        'ECOLOGIA': ['ECOLOGIA', 'ECOLOGÍA'],
        'ECONOMIA': ['ECONOM¿A'],
        'GEOLOGIA': ['GEOLOG¿A'],
        'GERENCIA': ['GERONTOLOGIA'],
        'INGENIERIA': ['INGENIER¿A', 'INGENIERÌA', 'INGENIIERIA', 'AGROINDUSTRIA'],
        'INSTRUMENTACION': ['INTRUMENTACION'],
        'MATEMATICAS': ['MATEMATICA'],
        'MARKETING': ['MERCADEO', 'MERCADOLOGIA'],
        'PSICOLOGIA': ['PSICOLOGA'],
        'QUIMICA': ['QUIMICA.', 'QU¿MICA'],
        'MUSICA': ['MUSICA-'],
        'TERAPIAS': ['TERAPIA', 'TERAPIAS'],
        'GESTION': ['GESTI¿N'],
        'NEGOCIOS': ['3°'],
        'DEPORTE': ['ACTIVIDAD'],
        'LICENCIATURA': ['Licenciatura', 'LICENC.EN'],
        'LENGUAS': ['LENGUAJES']
    }

    for key, values in reemplazos.items():
        df['ESTU_PRGM_ACADEMICO'] = df['ESTU_PRGM_ACADEMICO'].replace(values, key)

    # Eliminar columnas innecesarias
    df = df.drop(labels=['ESTU_PRIVADO_LIBERTAD', 'FAMI_TIENEINTERNET.1'], axis=1)

    # Renombrar columnas si es necesario
    if 'RENDIMIENTO_GLOBAL' in df.columns:
        df.columns = ['ID', 'PERIODO', 'PROGRAMA', 'DEPARTAMENTO', 'VALOR_MATRICULA', 'HORAS_TRABAJA', 'ESTRATO', 'TIENE_INTERNET', 'EDUCACION_PADRE', 'TIENE_LAVADORA', 'TIENE_AUTO', 'PAGA_MATRICULA', 'TIENE_COMPUTADOR', 'EDUCACION_MADRE', 'RENDIMIENTO_GLOBAL']
    else:
        df.columns = ['ID', 'PERIODO', 'PROGRAMA', 'DEPARTAMENTO', 'VALOR_MATRICULA', 'HORAS_TRABAJA', 'ESTRATO', 'TIENE_INTERNET', 'EDUCACION_PADRE', 'TIENE_LAVADORA', 'TIENE_AUTO', 'PAGA_MATRICULA', 'TIENE_COMPUTADOR', 'EDUCACION_MADRE']

    # Convertir valores de 'Si'/'No' a binario
    def convertir_num(valor):
        if valor == 'Si':
            return 1
        elif valor == 'No':
            return 0
        else:
            return valor

    df = df.applymap(convertir_num)

    # Imputar valores nulos en columnas categóricas con 'Desconocido'
    categorical_columns = ["VALOR_MATRICULA",'PERIODO', 'ESTRATO',"PROGRAMA", "DEPARTAMENTO", "EDUCACION_PADRE", "EDUCACION_MADRE"]
    for col in categorical_columns:
        df[col].fillna('Desconocido', inplace=True)

    # Imputar valores nulos en columnas numéricas con la media
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    for col in numerical_columns:
        df[col].fillna(df[col].mean(), inplace=True)

    # Convertir a categorías las columnas necesarias
    df[categorical_columns] = df[categorical_columns].astype("category")

    # Reemplazar valores de rendimiento
    valores_rendimiento = {'alto': 4, 'medio-alto': 3, 'medio-bajo': 2, 'bajo': 1}
    df = df.replace(valores_rendimiento)

    # Aplicar One-Hot Encoding a columnas categóricas específicas
    encoder = OneHotEncoder(sparse_output=False)

    encoded_columns = encoder.fit_transform(df[['HORAS_TRABAJA', 'PERIODO', 'VALOR_MATRICULA', 'PROGRAMA', 'DEPARTAMENTO', 'ESTRATO', 'EDUCACION_PADRE', 'EDUCACION_MADRE']])

    feature_names = encoder.get_feature_names_out(['HORAS_TRABAJA', 'PERIODO', 'VALOR_MATRICULA', 'PROGRAMA', 'DEPARTAMENTO', 'ESTRATO', 'EDUCACION_PADRE', 'EDUCACION_MADRE'])

    one_hot_encoded_df = pd.DataFrame(encoded_columns, columns=feature_names)

    # Eliminar columnas originales que fueron codificadas
    df.drop(labels=['HORAS_TRABAJA', 'PERIODO', 'VALOR_MATRICULA', 'PROGRAMA', 'DEPARTAMENTO', 'ESTRATO', 'EDUCACION_PADRE', 'EDUCACION_MADRE'], axis=1, inplace=True)

    # Concatenar el DataFrame original con el DataFrame codificado
    df = pd.concat([df, one_hot_encoded_df], axis=1)

    # Verificar si hay valores nulos en el DataFrame final
    print(df.isnull().sum())

    return df


In [ ]:
df = Preprocesar(df)
test_df = Preprocesar(test_df)

ID                                                      0
TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
PAGA_MATRICULA                                          0
                                                       ..
EDUCACION_MADRE_Primaria incompleta                     0
EDUCACION_MADRE_Secundaria (Bachillerato) completa      0
EDUCACION_MADRE_Secundaria (Bachillerato) incompleta    0
EDUCACION_MADRE_Técnica o tecnológica completa          0
EDUCACION_MADRE_Técnica o tecnológica incompleta        0
Length: 204, dtype: int64
ID                                                      0
TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
PAGA_MATRICULA                                          0
                                              

In [ ]:
df

,ID,TIENE_INTERNET,TIENE_LAVADORA,TIENE_AUTO,PAGA_MATRICULA,TIENE_COMPUTADOR,RENDIMIENTO_GLOBAL,HORAS_TRABAJA_0,HORAS_TRABAJA_Entre 11 y 20 horas,HORAS_TRABAJA_Entre 21 y 30 horas,...,EDUCACION_MADRE_Ninguno,EDUCACION_MADRE_No Aplica,EDUCACION_MADRE_No sabe,EDUCACION_MADRE_Postgrado,EDUCACION_MADRE_Primaria completa,EDUCACION_MADRE_Primaria incompleta,EDUCACION_MADRE_Secundaria (Bachillerato) completa,EDUCACION_MADRE_Secundaria (Bachillerato) incompleta,EDUCACION_MADRE_Técnica o tecnológica completa,EDUCACION_MADRE_Técnica o tecnológica incompleta
0,904256,1.0,1.0,1.0,0.0,1.0,3,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,645256,0.0,1.0,0.0,0.0,1.0,1,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,308367,1.0,1.0,0.0,0.0,0.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,470353,1.0,1.0,0.0,0.0,1.0,4,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
4,989032,1.0,1.0,1.0,0.0,1.0,2,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
692495,25096,1.0,1.0,0.0,1.0,1.0,3,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
692496,754213,1.0,1.0,0.0,0.0,1.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
692497,504185,1.0,1.0,0.0,1.0,1.0,2,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
692498,986620,0.0,0.0,0.0,1.0,1.0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
category_cols = [i for i in df.columns if not i in df._get_numeric_data()]
for c in category_cols:
    print ("%10s"%c, np.unique(df[c].dropna()))

RENDIMIENTO_GLOBAL ['alto' 'bajo' 'medio-alto' 'medio-bajo']


In [ ]:
print(df.isnull().sum())

ID                                                      0
ESTRATO                                                 0
TIENE_INTERNET                                          0
TIENE_LAVADORA                                          0
TIENE_AUTO                                              0
                                                       ..
EDUCACION_MADRE_Primaria incompleta                     0
EDUCACION_MADRE_Secundaria (Bachillerato) completa      0
EDUCACION_MADRE_Secundaria (Bachillerato) incompleta    0
EDUCACION_MADRE_Técnica o tecnológica completa          0
EDUCACION_MADRE_Técnica o tecnológica incompleta        0
Length: 197, dtype: int64


In [ ]:
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 MB 9.9 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import train_test_split

X = df.drop(labels =["RENDIMIENTO_GLOBAL",'ID'], axis=1)  # Características
y = df["RENDIMIENTO_GLOBAL"]  # Variable objetivo

# Dividir los datos en conjunto de entrenamiento y conjunto de prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1,random_state = 42)

# Crear el modelo CatBoost
catboost_model = CatBoostClassifier(iterations=2000, learning_rate=0.1, depth=6, loss_function='MultiClass')

# Entrenar el modelo
catboost_model.fit(X_train, y_train)

# Evaluar el modelo
accuracy = catboost_model.score(X_test, y_test)
print("Accuracy:", accuracy)


0:	learn: 1.3694010	total: 354ms	remaining: 11m 47s
1:	learn: 1.3558743	total: 732ms	remaining: 12m 11s
2:	learn: 1.3453566	total: 1.06s	remaining: 11m 42s
3:	learn: 1.3361408	total: 1.38s	remaining: 11m 26s
4:	learn: 1.3281485	total: 1.74s	remaining: 11m 33s
5:	learn: 1.3215930	total: 2.06s	remaining: 11m 25s
6:	learn: 1.3159744	total: 2.4s	remaining: 11m 24s
7:	learn: 1.3109502	total: 2.72s	remaining: 11m 17s
8:	learn: 1.3067082	total: 3.04s	remaining: 11m 12s
9:	learn: 1.3028659	total: 3.35s	remaining: 11m 6s
10:	learn: 1.2993714	total: 3.69s	remaining: 11m 6s
11:	learn: 1.2961281	total: 4.05s	remaining: 11m 11s
12:	learn: 1.2929949	total: 4.38s	remaining: 11m 9s
13:	learn: 1.2902609	total: 4.71s	remaining: 11m 7s
14:	learn: 1.2877290	total: 5.03s	remaining: 11m 5s
15:	learn: 1.2852381	total: 5.34s	remaining: 11m 1s
16:	learn: 1.2829693	total: 5.68s	remaining: 11m 2s
17:	learn: 1.2810612	total: 6.04s	remaining: 11m 5s
18:	learn: 1.2794211	total: 6.32s	remaining: 10m 59s
19:	learn: 1

In [ ]:
predictions = catboost_model.predict(test_df.drop('ID', axis=1))

predictions = predictions.flatten()

submit = pd.DataFrame({'ID': test_df['ID'], 'RENDIMIENTO_GLOBAL': predictions})

submit['RENDIMIENTO_GLOBAL'] = submit['RENDIMIENTO_GLOBAL'].replace({
    1: 'bajo',
    2: 'medio-bajo',
    3: 'medio-alto',
    4: 'alto'
})

submit.to_csv('submission.csv', index=False)